# 1. Photos geo-data frame

In [ ]:
from os import environ
from urllib import request

import dotenv
import exif
import geopandas as gpd
import numpy as np
import osmnx as ox
import requests
from plum.exceptions import UnpackError
from shapely import geometry
from tqdm import tqdm

tqdm.pandas()

In [ ]:
BASE_URI = "https://sensecity-africa.io"
CRS = "epsg:4326"

# find .env automagically by walking up directories until it's found, then load
# up the .env entries as environment variables
dotenv.load_dotenv(dotenv.find_dotenv())

username = environ.get("USERNAME")
password = environ.get("PASSWORD")

In [ ]:
auth_token = requests.post(
    f"{BASE_URI}/api-token-auth/",
    data=dict(username=username, password=password),
).json()["token"]

In [ ]:
photos_json = requests.get(
    f"{BASE_URI}/api/photos/",
    headers=dict(authorization=f"Token {auth_token}"),
).json()

In [ ]:
photos_gdf = gpd.GeoDataFrame(photos_json).dropna()
# ACHTUNG: need to flip the x, y coordinates of the API to have latitude/longitude of
# Nominatim, EXIF etc
photos_gdf["geometry"] = (
    photos_gdf["location"]
    .apply(geometry.shape)
    .apply(lambda p: geometry.Point(p.y, p.x))
)
# photos_gdf = photos_gdf.set_geometry("location")
photos_gdf.crs = CRS

In [ ]:
# total number of images
print(len(photos_gdf))

1022


In [ ]:
def dist_to_nominatim(gdf):
    gdf_proj = ox.project_gdf(gdf)
    nominatim_point = gpd.GeoSeries(
        [geometry.Point(ox.geocode(gdf.name))], crs=CRS
    ).to_crs(gdf_proj.crs)
    return gdf_proj["geometry"].apply(lambda p: nominatim_point.distance(p))

In [ ]:
photos_gdf["query"] = photos_gdf.apply(
    lambda gdf: f"{gdf['city']}, {gdf['country']}", axis=1
)
photos_gdf["dist_to_nominatim"] = photos_gdf.groupby("query").progress_apply(
    dist_to_nominatim
)

  0%|                                                                  | 0/47 [00:00<?, ?it/s]

  4%|██▍                                                       | 2/47 [00:00<00:03, 13.72it/s]

 17%|█████████▊                                                | 8/47 [00:00<00:01, 35.34it/s]

 30%|████████████████▉                                        | 14/47 [00:00<00:00, 42.92it/s]

 43%|████████████████████████▎                                | 20/47 [00:00<00:00, 46.27it/s]

 55%|███████████████████████████████▌                         | 26/47 [00:00<00:00, 49.69it/s]

 68%|██████████████████████████████████████▊                  | 32/47 [00:00<00:00, 49.10it/s]

 81%|██████████████████████████████████████████████           | 38/47 [00:00<00:00, 50.95it/s]

 94%|█████████████████████████████████████████████████████▎   | 44/47 [00:01<00:00, 43.39it/s]

100%|█████████████████████████████████████████████████████████| 47/47 [00:01<00:00, 43.54it/s]

In [ ]:
# number of images that have the nominatim-geocoded location
nominatim_geocoded = np.isclose(photos_gdf["dist_to_nominatim"], 0)
print(nominatim_geocoded.sum())

636


In [ ]:
# Credits to https://bit.ly/36iyni5
def decimal_coords(coords, ref):
    decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
    if ref == "S" or ref == "W":
        decimal_degrees = -decimal_degrees
    return decimal_degrees


def get_exif_geom(img_url):
    with request.urlopen(img_url) as src:
        try:
            img = exif.Image(src)
        except UnpackError:
            return None

    if img.has_exif:
        try:
            return geometry.Point(
                decimal_coords(img.gps_latitude, img.gps_latitude_ref),
                decimal_coords(img.gps_longitude, img.gps_longitude_ref),
            )
        except AttributeError:
            pass

    return None

In [ ]:
# photos_gdf.loc[nominatim_geocoded, "exif_loc"] = photos_gdf[nominatim_geocoded][
#     "image"
# ].progress_apply(get_exif_location)
photos_gdf["exif_geom"] = photos_gdf["image"].progress_apply(get_exif_geom)

  0%|                                                                | 0/1022 [00:00<?, ?it/s]

  0%|                                                        | 2/1022 [00:00<02:31,  6.73it/s]

  0%|▏                                                       | 3/1022 [00:00<03:59,  4.26it/s]

  0%|▏                                                       | 4/1022 [00:01<04:47,  3.54it/s]

  0%|▎                                                       | 5/1022 [00:01<05:45,  2.95it/s]

...

100%|████████████████████████████████████████████████████▊| 1019/1022 [04:49<00:00,  3.58it/s]

100%|████████████████████████████████████████████████████▉| 1020/1022 [04:49<00:00,  3.57it/s]

100%|████████████████████████████████████████████████████▉| 1021/1022 [04:49<00:00,  3.59it/s]

100%|█████████████████████████████████████████████████████| 1022/1022 [04:49<00:00,  3.62it/s]

100%|█████████████████████████████████████████████████████| 1022/1022 [04:50<00:00,  3.52it/s]

In [ ]:
# number of images with the nominatim-geocoded location that have EXIF location data
print((~photos_gdf.loc[nominatim_geocoded, "exif_geom"].isna()).sum())

121


In [ ]:
dst_filepath = "../data/processed/photos.gpkg"

In [ ]:
photos_gdf["tags"] = photos_gdf["tags"].apply(lambda l: ", ".join(l))
photos_gdf["exif_lat"] = photos_gdf["exif_geom"].apply(
    lambda p: getattr(p, "x", np.nan)
)
photos_gdf["exif_lon"] = photos_gdf["exif_geom"].apply(
    lambda p: getattr(p, "y", np.nan)
)
photos_gdf.drop("exif_geom", axis=1).to_file(dst_filepath)

/home/martibosch/mambaforge/envs/sensecity-africa/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
